In [2]:
import h5py
from itertools import product
import pandas as pd
import numpy as np
import requests

In [3]:
baseUrl = 'http://www.illustris-project.org/api/'
headers = {"api-key":"7a2f32cbf6bba4a993ce194c6ca7b7be"}
def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

     # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
         return r.json() # parse json responses automatically
    return r

In [4]:
r = get(baseUrl)
names = [sim['name'] for sim in r['simulations']]
i = names.index('Illustris-1')
sim = get( r['simulations'][i]['url'] )
snaps = get( sim['snapshots'] )
snap = get( snaps[-1]['url'] )
snap

{'simulation': 'http://www.illustris-project.org/api/Illustris-1/',
 'number': 135,
 'redshift': 2.2204460492503099e-16,
 'num_gas': 5268982240,
 'num_dm': 6028568000,
 'num_trmc': 6028568000,
 'num_stars': 606875892,
 'num_bhs': 32552,
 'num_groups_fof': 7713601,
 'num_groups_subfind': 4366546,
 'num_groups_rockstar': 0,
 'filesize_snapshot': 1641304217072.0,
 'filesize_groupcat': 4812516112.0,
 'filesize_rockstar': 0.0,
 'url': 'http://www.illustris-project.org/api/Illustris-1/snapshots/135/',
 'subhalos': 'http://www.illustris-project.org/api/Illustris-1/snapshots/135/subhalos/',
 'vis': {'groupcat_sfms': 'http://www.illustris-project.org/api/Illustris-1/snapshots/135/subhalos/plot.png?xQuant=mstar2_log&yQuant=ssfr&xlim=9.0%2C11.0',
  'groupcat_sizemass': 'http://www.illustris-project.org/api/Illustris-1/snapshots/135/subhalos/plot.png?xQuant=mstar2_log&yQuant=size_stars',
  'groupcat_mz_gas': 'http://www.illustris-project.org/api/Illustris-1/snapshots/135/subhalos/plot.png?xQuant=m

In [5]:
subs = get( snap['subhalos'] )

In [6]:
def get(path, params=None):
     # make HTTP GET request to path
     r = requests.get(path, params=params, headers=headers)

     # raise exception if response code is not HTTP SUCCESS (200)
     r.raise_for_status()

     if r.headers['content-type'] == 'application/json':
         return r.json() # parse json responses automatically

     if 'content-disposition' in r.headers:
         filename = r.headers['content-disposition'].split("filename=")[1]
         with open(filename, 'wb') as f:
             f.write(r.content)
         return filename # return the filename string

     return r

# Cuentas Subhalos

In [7]:
cd cosmo_full

In [ ]:
e=0.0001

output_grid='Full_zeros.npy'
#raw_data='cosmo_fullm'
raw_data = subs

out_data='cosmo_full/arrays/'


def process(postable):
    postable['x_b']=np.floor(postable['x']/((75000+e)/1024))
    postable['y_b']=np.floor(postable['y']/((75000+e)/1024))
    postable['z_b']=np.floor(postable['z']/((75000+e)/1024))
    postable['x_b'] = postable['x_b'].astype(int)
    postable['y_b'] = postable['y_b'].astype(int)
    postable['z_b'] = postable['z_b'].astype(int)
    return postable

np.save(output_grid,np.zeros((1024,1024,1024)))

for i in range(1):
    filename = get(get(raw_data['results'][i]['url'])['trees']['sublink_mpb'])
    f = h5py.File(filename, 'r')
    position=np.array(f['SubhaloPos'])
    if len(position)==0:
        pass
    postable=pd.DataFrame(position)
    postable.columns=(['x','y','z'])
    mergetable=process(postable) 
    mergetable['c']=1
    def dataframe_to_array(df, out_shp):
        ids = np.ravel_multi_index(df[['x_b','y_b','z_b']].values.T, out_shp)
        val = df['count'].values
        return np.bincount(ids, val, minlength=np.prod(out_shp)).reshape(out_shp)

    counts=mergetable.groupby(['x_b','y_b','z_b'])['c'].count().reset_index(name="count")
    arr=dataframe_to_array(counts, (1024,1024,1024))
    old_arr=np.load(output_grid)
    np.save(output_grid,old_arr+arr)
    print('finished: '+filename)



pos=list(np.arange(0,1024,32))
ranges=list(product(pos,repeat=3))
print('Generating subboxes....')
out = np.load(output_grid)
for ID in ranges:
    f_box=out[ID[0]:ID[0]+32,ID[1]:ID[1]+32,ID[2]:ID[2]+32]
    np.save(out_data+str(ID[0])+'_'+str(ID[1])+'_'+str(ID[2])+'.npy',f_box)
print('All finished')